# Scraping geographical coordinates

In this section we import some information about Swiss cities (geo-coordinates, population) using a Google API.

First parsing of geographical coordinates. WARN: invalid key reported. The output is a *geography.csv* file, containing the cities' name and coordinates (latitude, longitude).

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

data = pd.read_csv('lausanne_marathon_2016.csv')

In [8]:
dictionary = {}
cities = data.lieu.value_counts().index
import googlemaps
gmaps = googlemaps.Client(key='***')

In [1]:
for city in cities:
    geocode_result = gmaps.geocode(city)
    if (len(geocode_result)>0):
        coord = [geocode_result[0]['geometry']['location']['lat'],geocode_result[0]['geometry']['location']['lng']]
        dictionary[city]=coord
    else:
        dictionary[city]='nan'

In [59]:
latitude = []
longitude = []
for city in cities:
    latitude.append(dictionary[city][0])
    longitude.append(dictionary[city][1])
geography = pd.DataFrame({'city' : cities, 'latitude' : latitude, 'longitude' : longitude})
geography.to_csv('geography.csv')

Add to the dataset the number of runners participating to the Lausanne's Marathon as an additional column and save into *geoinfo.csv*.

In [53]:
cities = pd.read_csv('geography.csv',encoding = 'latin1')
del cities['Unnamed: 0']

In [54]:
runners_per_city = data.lieu.value_counts()
population = []
for city in cities.city:
    k = 0
    while (k < len(cities)) and (runners_per_city.index[k]!=city):
        k +=1
    population.append(runners_per_city[runners_per_city.index[k]])

In [55]:
cities['runners']=population

In [56]:
cities.to_csv('geoinfo.csv',encoding='latin1')

# Scraping geographical distance from Lausanne

Using again a Google API, we save into a suitable file *distances.csv* the real distance [m] between the runner's city and Lausanne.

In [33]:
distance = []
for city in cities:
    d = gmaps.distance_matrix(city,'Lausanne')
    if d['rows'][0]['elements'][0]['status']=='OK':
        distance.append(d['rows'][0]['elements'][0]['distance']['value'])
    else:
        distance.append('nan')
len(distance)

2005

In [48]:
df = pd.DataFrame({'city' : cities, 'distance' : distance})
df.to_csv('distances.csv',encoding='latin1')

# Merge datasets

Merge the two datasets *geoinfo.csv* and *distances.csv* into a unique file *geodistance.csv*.

In [73]:
cities = pd.read_csv('geoinfo.csv',encoding='latin1')
distances = pd.read_csv('distances.csv',encoding='latin1')
del cities['Unnamed: 0']
del distances['Unnamed: 0']

In [76]:
dataset = pd.merge(cities,distances,on='city')
dataset.to_csv('geodistance.csv',encoding='latin1')

# Scraping population data

In this section we try to map each city contained in geodistance.csv with its overall population. We scrape the population information from the following Wikipedia [page](https://en.wikipedia.org/wiki/List_of_cities_in_Switzerland) using the *Requests* library.

In [78]:
geoinfo = pd.read_csv('geodistance.csv',encoding='latin1')
del geoinfo['Unnamed: 0']
geoinfo.head(5)

,city,latitude,longitude,runners,distance
0,St-Légier-La Chiésaz,46.47171849999999,6.876770899999999,12,24209.0
1,St-Légier,46.47171849999999,6.876770899999999,57,24023.0
2,Ernen,46.39850010000001,8.1457733,1,165658.0
3,Lausanne,46.5196535,6.6322734,2043,0.0
4,Pully,46.5092681,6.6654949,312,3001.0


In [79]:
import requests
url = 'https://en.wikipedia.org/wiki/List_of_cities_in_Switzerland'
r = requests.get(url)
table = pd.read_html(r.text)
table = table[1]

We are interested in the second table, which provides the relevant information. We import city name, Canton and Population for eventual visualization purposes.

In [80]:
city = table[0][2:]
city = city.reset_index(range(len(city)))
city = city.rename(columns={0 : 'name'})
del city['index']
canton = table[3][2:]
canton = canton.reset_index(range(len(city)))
canton = canton.rename(columns={3 : 'canton'})
del canton['index']
population = table[4][2:]
population = population.reset_index(range(len(city)))
population = population.rename(columns={4 : 'population'})
del population['index']

pop = pd.DataFrame()
pop['city']=city
pop['canton']=canton
pop['population']=population

In [81]:
pop.head()

,city,canton,population
0,Aarau,AG,20710
1,Aarberg,BE,4529
2,Aarburg,AG,7723
3,Adliswil,ZH,18551
4,Aesch (BL)[note 1],BL,10198


Try to link each city in *cities* with the names in *pop* through a suitable dictionary:

In [82]:
mapping = {}
for city in geoinfo.city:
    k = 0
    length = len(pop)
    while (k < length) and ((city in pop.city[k])!=True):
        k = k+1
    if k==length:
        mapping[city]='nan'
    else:
        mapping[city]=k

In [83]:
population = []
canton = []
for city in geoinfo.city:
    if mapping[city]=='nan':
        population.append('nan')
        canton.append('nan')
    else:
        population.append(pop.population[mapping[city]])
        canton.append(pop.canton[mapping[city]])
geoinfo['population']=population
geoinfo['canton']=canton

In [84]:
geoinfo.to_csv('complete_geography.csv',encoding='latin1')